In [1]:
# SETUP CELL (run first)
import os
import glob
import pickle
from datetime import datetime

from pymongo import MongoClient
from bson import json_util

# 1) Unzip dataset (creates sample_mflix/ directory)
# If you already unzipped, it's safe to re-run (it may overwrite files)
!unzip -o sample_mflix.zip -d sample_mflix

# 2) Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["sample_mflix"]

# 3) Helper: ensure answers folder exists
os.makedirs("answers", exist_ok=True)

def save_answer(qnum, obj):
    """Pickle results to answers/q<N>.pkl exactly as required."""
    with open(f"answers/q{qnum}.pkl", "wb") as f:
        pickle.dump(obj, f)

def load_json_file_to_collection(path, collection_name):
    """
    Loads a JSON file into db[collection_name] using bson.json_util.loads.
    Handles either:
      - a JSON array (most common)
      - a single JSON object
      - line-delimited JSON (fallback)
    """
    with open(path, "r", encoding="utf-8") as f:
        text = f.read().strip()

    col = db[collection_name]
    col.drop()  # start fresh so reruns don't duplicate

    # Try full-file parse first
    try:
        data = json_util.loads(text)
        if isinstance(data, list):
            if data:
                col.insert_many(data)
        elif isinstance(data, dict):
            col.insert_one(data)
        else:
            # unexpected, fallback to line-delimited
            raise ValueError("Unexpected JSON root type")
        return

    except Exception:
        # Fallback: line-delimited JSON
        docs = []
        with open(path, "r", encoding="utf-8") as f2:
            for line in f2:
                line = line.strip()
                if not line:
                    continue
                docs.append(json_util.loads(line))
        if docs:
            col.insert_many(docs)

# 4) Load all JSON files into collections
json_dir = "sample_mflix"
json_paths = sorted(glob.glob(os.path.join(json_dir, "*.json")))

print("Found JSON files:", [os.path.basename(p) for p in json_paths])

for p in json_paths:
    name = os.path.splitext(os.path.basename(p))[0]  # filename without .json
    load_json_file_to_collection(p, name)
    print(f"Loaded {name}: {db[name].count_documents({})} docs")

# 5) Create geospatial index for theaters (needed for Q32-Q34)
# (safe to run even if it already exists)
if "theaters" in db.list_collection_names():
    db.theaters.create_index([("location.geo", "2dsphere")])

print("Collections:", db.list_collection_names())


Archive:  sample_mflix.zip
  inflating: sample_mflix/__MACOSX/._sample_mflix  
  inflating: sample_mflix/sample_mflix/sessions.json  
  inflating: sample_mflix/__MACOSX/sample_mflix/._sessions.json  
  inflating: sample_mflix/sample_mflix/movies.json  
  inflating: sample_mflix/__MACOSX/sample_mflix/._movies.json  
  inflating: sample_mflix/sample_mflix/comments.json  
  inflating: sample_mflix/__MACOSX/sample_mflix/._comments.json  
  inflating: sample_mflix/sample_mflix/theaters.json  
  inflating: sample_mflix/__MACOSX/sample_mflix/._theaters.json  
  inflating: sample_mflix/sample_mflix/users.json  
  inflating: sample_mflix/__MACOSX/sample_mflix/._users.json  
Found JSON files: []
Collections: []


In [2]:
import os
os.getcwd()


'/Users/saifyfairozkhan/CS-639-Data_mgmt_practice-/p2'

In [3]:
json_dir = "sample_mflix"
json_paths = sorted(glob.glob(os.path.join(json_dir, "*.json")))


In [4]:
# 4) Load all JSON files into collections
json_dir = os.path.join("sample_mflix", "sample_mflix")  # <-- nested folder
json_paths = sorted(glob.glob(os.path.join(json_dir, "*.json")))


In [5]:
# 4) Load all JSON files into collections (robust)
json_paths = sorted(glob.glob("sample_mflix/**/*.json", recursive=True))

# ignore __MACOSX and resource fork files
json_paths = [
    p for p in json_paths
    if "__MACOSX" not in p and not os.path.basename(p).startswith("._")
]

print("Found JSON files:", [os.path.basename(p) for p in json_paths])


Found JSON files: ['comments.json', 'movies.json', 'sessions.json', 'theaters.json', 'users.json']


In [7]:
for p in json_paths:
    name = os.path.splitext(os.path.basename(p))[0]  # comments, movies, users, ...
    load_json_file_to_collection(p, name)
    print(f"Loaded {name}: {db[name].count_documents({})} docs")

print("Collections:", db.list_collection_names())

# geospatial index for later (Q32-Q34)
if "theaters" in db.list_collection_names():
    db.theaters.create_index([("location.geo", "2dsphere")])


Loaded comments: 50304 docs
Loaded movies: 23539 docs
Loaded sessions: 1 docs
Loaded theaters: 1564 docs
Loaded users: 185 docs
Collections: ['sessions', 'movies', 'theaters', 'users', 'comments']


In [8]:
import os
os.makedirs("answers", exist_ok=True)


In [10]:
#q1
q1 = [db.movies.find_one({})]
with open("answers/q1.pkl", "wb") as f:
    pickle.dump(q1, f)
q1


[{'_id': ObjectId('573a1390f29313caabcd4135'),
  'plot': 'Three men hammer on an anvil and pass a bottle of beer around.',
  'genres': ['Short'],
  'runtime': 1,
  'cast': ['Charles Kayser', 'John Ott'],
  'num_mflix_comments': 1,
  'title': 'Blacksmith Scene',
  'fullplot': 'A stationary camera looks at a large anvil with a blacksmith behind it and one on either side. The smith in the middle draws a heated metal rod from the fire, places it on the anvil, and all three begin a rhythmic hammering. After several blows, the metal goes back in the fire. One smith pulls out a bottle of beer, and they each take a swig. Then, out comes the glowing metal and the hammering resumes.',
  'countries': ['USA'],
  'released': datetime.datetime(1893, 5, 9, 0, 0),
  'directors': ['William K.L. Dickson'],
  'rated': 'UNRATED',
  'awards': {'wins': 1, 'nominations': 0, 'text': '1 win.'},
  'lastupdated': '2015-08-26 00:03:50.133000000',
  'year': 1893,
  'imdb': {'rating': 6.2, 'votes': 1189, 'id': 5},


In [11]:
#q2
q2 = list(db.movies.find({"directors": "Christopher Nolan"}))
with open("answers/q2.pkl", "wb") as f:
    pickle.dump(q2, f)
len(q2)


9

In [12]:
#q3
q3 = list(db.users.find({}).limit(5))
with open("answers/q3.pkl", "wb") as f:
    pickle.dump(q3, f)
q3


[{'_id': ObjectId('59b99db4cfa9a34dcd7885b6'),
  'name': 'Ned Stark',
  'email': 'sean_bean@gameofthron.es',
  'password': '$2b$12$UREFwsRUoyF0CRqGNK0LzO0HM/jLhgUCNNIJ9RJAqMUQ74crlJ1Vu'},
 {'_id': ObjectId('59b99db4cfa9a34dcd7885b7'),
  'name': 'Robert Baratheon',
  'email': 'mark_addy@gameofthron.es',
  'password': '$2b$12$yGqxLG9LZpXA2xVDhuPnSOZd.VURVkz7wgOLY3pnO0s7u2S1ZO32y'},
 {'_id': ObjectId('59b99db5cfa9a34dcd7885b8'),
  'name': 'Jaime Lannister',
  'email': 'nikolaj_coster-waldau@gameofthron.es',
  'password': '$2b$12$6vz7wiwO.EI5Rilvq1zUc./9480gb1uPtXcahDxIadgyC3PS8XCUK'},
 {'_id': ObjectId('59b99db5cfa9a34dcd7885b9'),
  'name': 'Catelyn Stark',
  'email': 'michelle_fairley@gameofthron.es',
  'password': '$2b$12$fiaTH5Sh1zKNFX2i/FTEreWGjxoJxvmV7XL.qlfqCr8CwOxK.mZWS'},
 {'_id': ObjectId('59b99db6cfa9a34dcd7885ba'),
  'name': 'Cersei Lannister',
  'email': 'lena_headey@gameofthron.es',
  'password': '$2b$12$FExjgr7CLhNCa.oUsB9seub8mqcHzkJCFZ8heMc8CeIKOZfeTKP8m'}]

In [13]:
#q4
q4 = [db.movies.find_one({"imdb.rating": {"$gt": 9}})]
with open("answers/q4.pkl", "wb") as f:
    pickle.dump(q4, f)
q4


[{'_id': ObjectId('573a1396f29313caabce4a9a'),
  'fullplot': 'When the aging head of a famous crime family decides to transfer his position to one of his subalterns, a series of unfortunate events start happening to the family, and a war begins between all the well-known families leading to insolence, deportation, murder and revenge, and ends with the favorable successor being finally chosen.',
  'imdb': {'rating': 9.2, 'votes': 1038358, 'id': 68646},
  'year': 1972,
  'plot': 'The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.',
  'genres': ['Crime', 'Drama'],
  'rated': 'R',
  'metacritic': 100,
  'title': 'The Godfather',
  'lastupdated': '2015-09-02 00:08:23.680000000',
  'languages': ['English', 'Italian', 'Latin'],
  'writers': ['Mario Puzo (screenplay)',
   'Francis Ford Coppola (screenplay)',
   'Mario Puzo (novel)'],
  'type': 'movie',
  'tomatoes': {'website': 'http://www.thegodfather.com',
   'viewer': {'rating

In [14]:
#q5
q5 = db.movies.count_documents({})
with open("answers/q5.pkl", "wb") as f:
    pickle.dump(q5, f)
q5


23539

In [15]:
import pickle
import os
os.makedirs("answers", exist_ok=True)


In [16]:
#q6
q6 = db.movies.count_documents({"year": {"$gt": 2000}})
with open("answers/q6.pkl", "wb") as f:
    pickle.dump(q6, f)
q6


13103

In [17]:
#q7
q7 = db.movies.count_documents({"genres": "Comedy"})
with open("answers/q7.pkl", "wb") as f:
    pickle.dump(q7, f)
q7


7024

In [18]:
#q8
q8 = db.comments.count_documents({"name": "Taylor Hill"})
with open("answers/q8.pkl", "wb") as f:
    pickle.dump(q8, f)
q8


279

In [19]:
#q9
q9 = db.movies.count_documents({"runtime": {"$gt": 120}})
with open("answers/q9.pkl", "wb") as f:
    pickle.dump(q9, f)
q9


3722

In [20]:
#q10
q10 = list(
    db.movies.find(
        {"year": 2015},
        {"_id": 0, "title": 1}
    ).sort("title", 1)
)
with open("answers/q10.pkl", "wb") as f:
    pickle.dump(q10, f)
q10[:5], len(q10)


([{'title': '(T)ERROR'},
  {'title': '11 Minutes'},
  {'title': '13 Minutes'},
  {'title': '3 1/2 Minutes, Ten Bullets'},
  {'title': '45 Years'}],
 484)

In [21]:
#q11
q11 = list(db.comments.find({}).sort("date", -1).limit(5))
with open("answers/q11.pkl", "wb") as f:
    pickle.dump(q11, f)
q11


[{'_id': ObjectId('5b7327c34a68c3f7a7bcb1d3'),
  'name': 'sxkybvzb6',
  'email': 'vtz4prjvb@g622o.1yk',
  'movie_id': ObjectId('573a13eff29313caabdd82f3'),
  'text': 'fazzlebizzle',
  'date': datetime.datetime(2018, 8, 14, 15, 5, 3, 142000)},
 {'_id': ObjectId('5b7327724a68c3f7a7bcb1ae'),
  'name': 'jjjdmntww',
  'email': 'h90xxa9b0@gn3q7.7th',
  'movie_id': ObjectId('573a13eff29313caabdd82f3'),
  'text': 'feefee',
  'date': datetime.datetime(2018, 8, 14, 15, 3, 14, 306000)},
 {'_id': ObjectId('5b7326fd4a68c3f7a7bcb188'),
  'name': 'yznjm1yul',
  'email': 'rio72go4u@wztfh.ep0',
  'movie_id': ObjectId('573a13eff29313caabdd82f3'),
  'text': 'feefee',
  'date': datetime.datetime(2018, 8, 14, 15, 1, 16, 716000)},
 {'_id': ObjectId('5b7326bd4a68c3f7a7bcb15f'),
  'name': 'sg1u6swo4',
  'email': 'gs4jr54ar@6gvpw.ok0',
  'movie_id': ObjectId('573a13eff29313caabdd82f3'),
  'text': 'fazzlebizzle',
  'date': datetime.datetime(2018, 8, 14, 15, 0, 25, 211000)},
 {'_id': ObjectId('5b7325024a68c3f7a7

In [22]:
#q12
q12 = db.movies.count_documents({"genres": {"$all": ["Action", "Adventure"]}})
with open("answers/q12.pkl", "wb") as f:
    pickle.dump(q12, f)
q12


779

In [23]:
#q13
q13 = list(
    db.movies.find(
        {"imdb.rating": {"$gte": 9, "$lte": 10}},
        {"_id": 0, "title": 1, "imdb.rating": 1, "genres": 1, "year": 1}
    )
)
with open("answers/q13.pkl", "wb") as f:
    pickle.dump(q13, f)
q13[:3], len(q13)


([{'imdb': {'rating': 9.2},
   'year': 1972,
   'genres': ['Crime', 'Drama'],
   'title': 'The Godfather'},
  {'imdb': {'rating': 9.1},
   'year': 1974,
   'genres': ['Crime', 'Drama'],
   'title': 'The Godfather: Part II'},
  {'genres': ['Drama', 'History'],
   'title': 'I, Claudius',
   'year': 1976,
   'imdb': {'rating': 9}}],
 31)

In [24]:
#q14
q14 = db.movies.count_documents({
    "directors": {"$exists": True},
    "$expr": {"$eq": [{"$size": "$directors"}, 3]}
})
with open("answers/q14.pkl", "wb") as f:
    pickle.dump(q14, f)
q14


181

In [25]:
#q15
q15 = db.movies.count_documents({
    "directors": {"$exists": True},
    "$expr": {"$gte": [{"$size": "$directors"}, 3]}
})
with open("answers/q15.pkl", "wb") as f:
    pickle.dump(q15, f)
q15


296

In [26]:
#q16
pipeline16 = [
    {"$lookup": {
        "from": "movies",
        "localField": "movie_id",
        "foreignField": "_id",
        "as": "movie"
    }},
    {"$unwind": "$movie"},
    {"$match": {"movie.year": 2010}},
    {"$count": "total"}
]
res16 = list(db.comments.aggregate(pipeline16))
q16 = res16[0]["total"] if res16 else 0

with open("answers/q16.pkl", "wb") as f:
    pickle.dump(q16, f)
q16


510

In [27]:
#q17
pipeline17 = [
    {"$lookup": {
        "from": "movies",
        "localField": "movie_id",
        "foreignField": "_id",
        "as": "movie"
    }},
    {"$unwind": "$movie"},
    {"$match": {"movie.genres": "Action"}},
    {"$group": {"_id": None, "names": {"$addToSet": "$name"}}}
]
res17 = list(db.comments.aggregate(pipeline17))
q17 = sorted(res17[0]["names"]) if res17 else []

with open("answers/q17.pkl", "wb") as f:
    pickle.dump(q17, f)
q17[:10], len(q17)


(['Alliser Thorne',
  'Amy Phillips',
  'Amy Ramirez',
  'Andrea Le',
  'Anthony Cline',
  'Anthony Hurst',
  'Anthony Smith',
  'Anthony Thompson',
  'April Cole',
  'Arya Stark'],
 183)

In [28]:
#q18
from datetime import datetime

cutoff = datetime(2018, 8, 13)
q18 = list(db.comments.find({"date": {"$gte": cutoff}}))
with open("answers/q18.pkl", "wb") as f:
    pickle.dump(q18, f)
len(q18)


29

In [29]:
#q19
pipeline19 = [
    {"$lookup": {
        "from": "movies",
        "localField": "movie_id",
        "foreignField": "_id",
        "as": "movie"
    }},
    {"$unwind": "$movie"},
    {"$match": {"movie.directors": "Steven Spielberg"}},
    {"$count": "total"}
]
res19 = list(db.comments.aggregate(pipeline19))
q19 = res19[0]["total"] if res19 else 0

with open("answers/q19.pkl", "wb") as f:
    pickle.dump(q19, f)
q19


1269

In [30]:
#q20
q20 = list(
    db.movies.find(
        {"imdb.rating": {"$gt": 9.0}, "released": {"$exists": True}},
        {"_id": 0, "title": 1, "released": 1, "imdb.rating": 1}
    ).sort("released", -1).limit(10)
)
with open("answers/q20.pkl", "wb") as f:
    pickle.dump(q20, f)
q20


[{'title': 'A Brave Heart: The Lizzie Velasquez Story',
  'released': datetime.datetime(2015, 9, 25, 0, 0),
  'imdb': {'rating': 9.4}},
 {'title': 'The Real Miyagi',
  'released': datetime.datetime(2015, 2, 20, 0, 0),
  'imdb': {'rating': 9.3}},
 {'title': 'Over the Garden Wall',
  'released': datetime.datetime(2014, 11, 3, 0, 0),
  'imdb': {'rating': 9.2}},
 {'title': 'Frozen Planet',
  'released': datetime.datetime(2012, 3, 18, 0, 0),
  'imdb': {'rating': 9.2}},
 {'title': 'Human Planet',
  'released': datetime.datetime(2011, 1, 13, 0, 0),
  'imdb': {'rating': 9.2}},
 {'title': 'Life',
  'released': datetime.datetime(2009, 10, 12, 0, 0),
  'imdb': {'rating': 9.2}},
 {'title': 'Planet Earth',
  'released': datetime.datetime(2007, 3, 25, 0, 0),
  'imdb': {'rating': 9.5}},
 {'title': 'The Blue Planet',
  'released': datetime.datetime(2002, 1, 27, 0, 0),
  'imdb': {'rating': 9.2}},
 {'title': 'Band of Brothers',
  'released': datetime.datetime(2001, 9, 9, 0, 0),
  'imdb': {'rating': 9.6}